In [28]:
!pip install -q ultralytics

import os, glob, shutil, zipfile
from ultralytics import YOLO

print("✅ ultralytics installed")


✅ ultralytics installed


In [29]:
# Clean previous extractions (if any)
for d in [
    "/content/pheasants1200images",
    "/content/pheasants1200_labels",
    "/content/black_grouse_images",
    "/content/blackgrouse1200_labels",
    "/content/skylark_images",
    "/content/skylark1000_labels",
    "/content/woodlark_images",
    "/content/woodlark1000_labels",
    "/content/datasets",
]:
    shutil.rmtree(d, ignore_errors=True)

os.makedirs("/content", exist_ok=True)

def unzip_to(src_zip, dst_dir):
    assert os.path.exists(src_zip), f"Missing: {src_zip}"
    os.makedirs(dst_dir, exist_ok=True)
    with zipfile.ZipFile(src_zip, 'r') as z:
        z.extractall(dst_dir)
    print(f"📦 Unzipped {os.path.basename(src_zip)} → {dst_dir}")

# Pheasant
unzip_to("/content/pheasants1200images.zip", "/content/pheasants1200images")
unzip_to("/content/pheasants1200.zip",       "/content/pheasants1200_labels")

# Black grouse
unzip_to("/content/black_grouse_images.zip", "/content/black_grouse_images")
unzip_to("/content/blackgrouse1200.zip",     "/content/blackgrouse1200_labels")

# Skylark
unzip_to("/content/skylark(1).zip",          "/content/skylark_images")
unzip_to("/content/Skylark1000.zip",         "/content/skylark1000_labels")

# Woodlark
unzip_to("/content/woodlark.zip",            "/content/woodlark_images")
unzip_to("/content/woodlark1000.zip",        "/content/woodlark1000_labels")


📦 Unzipped pheasants1200images.zip → /content/pheasants1200images
📦 Unzipped pheasants1200.zip → /content/pheasants1200_labels
📦 Unzipped black_grouse_images.zip → /content/black_grouse_images
📦 Unzipped blackgrouse1200.zip → /content/blackgrouse1200_labels
📦 Unzipped skylark(1).zip → /content/skylark_images
📦 Unzipped Skylark1000.zip → /content/skylark1000_labels
📦 Unzipped woodlark.zip → /content/woodlark_images
📦 Unzipped woodlark1000.zip → /content/woodlark1000_labels


In [30]:
DATASET_ROOT = "/content/datasets/legal_illegal"
IMAGES_OUT = os.path.join(DATASET_ROOT, "images")
LABELS_OUT = os.path.join(DATASET_ROOT, "labels")

shutil.rmtree(DATASET_ROOT, ignore_errors=True)
os.makedirs(IMAGES_OUT, exist_ok=True)
os.makedirs(LABELS_OUT, exist_ok=True)

def gather_images(root):
    files = []
    for ext in ("*.jpg", "*.jpeg", "*.png"):
        files.extend(glob.glob(os.path.join(root, "**", ext), recursive=True))
    return files

def gather_labels(root):
    return glob.glob(os.path.join(root, "**", "*.txt"), recursive=True)

def add_species(image_root, label_root, species_prefix, mode):
    """
    mode:
      'keep'   -> keep original class index from txt
      'force0' -> all class = 0 (legal)
      'force1' -> all class = 1 (illegal)
    """
    img_files = gather_images(image_root)
    lbl_files = gather_labels(label_root)

    print(f"\n🔎 {species_prefix}: {len(img_files)} images, {len(lbl_files)} label files")

    # basename (without extension) -> image path
    img_map = {os.path.splitext(os.path.basename(p))[0]: p for p in img_files}

    added = 0
    for lbl_path in lbl_files:
        base = os.path.splitext(os.path.basename(lbl_path))[0]
        img_path = img_map.get(base)
        if not img_path:
            continue  # label with no matching image

        new_base = f"{species_prefix}_{base}"
        img_ext = os.path.splitext(img_path)[1]
        out_img = os.path.join(IMAGES_OUT, new_base + img_ext)
        out_lbl = os.path.join(LABELS_OUT, new_base + ".txt")

        shutil.copy2(img_path, out_img)

        with open(lbl_path, "r") as f:
            lines = [ln.strip() for ln in f if ln.strip()]

        new_lines = []
        for ln in lines:
            parts = ln.split()
            if not parts:
                continue
            if mode == "keep":
                cls = int(parts[0])
            elif mode == "force0":
                cls = 0
            elif mode == "force1":
                cls = 1
            else:
                raise ValueError("Unknown mode")
            parts[0] = str(cls)
            new_lines.append(" ".join(parts))

        with open(out_lbl, "w") as f:
            f.write("\n".join(new_lines))

        added += 1

    print(f"✅ {species_prefix}: added {added} image/label pairs")

# 1) Pheasant: images & labels
add_species(
    image_root="/content/pheasants1200images",
    label_root="/content/pheasants1200_labels/obj_Train_data",
    species_prefix="pheasant",
    mode="keep",
)

# 2) Black grouse
add_species(
    image_root="/content/black_grouse_images",
    label_root="/content/blackgrouse1200_labels/obj_train_data",
    species_prefix="blackgrouse",
    mode="keep",
)

# 3) Skylark: all legal
add_species(
    image_root="/content/skylark_images/skylark",
    label_root="/content/skylark1000_labels/obj_train_data/skylark",
    species_prefix="skylark",
    mode="force0",
)

# 4) Woodlark: all illegal
add_species(
    image_root="/content/woodlark_images/woodlark",
    label_root="/content/woodlark1000_labels/woodlark1000/obj_train_data/woodlark",
    species_prefix="woodlark",
    mode="force1",
)

print("\n📁 MERGE COMPLETE")
print("Total images:", len(os.listdir(IMAGES_OUT)))
print("Total labels:", len(os.listdir(LABELS_OUT)))



🔎 pheasant: 1200 images, 1200 label files
✅ pheasant: added 1200 image/label pairs

🔎 blackgrouse: 1200 images, 1200 label files
✅ blackgrouse: added 1200 image/label pairs

🔎 skylark: 1000 images, 1000 label files
✅ skylark: added 1000 image/label pairs

🔎 woodlark: 1000 images, 1000 label files
✅ woodlark: added 1000 image/label pairs

📁 MERGE COMPLETE
Total images: 4400
Total labels: 4400


In [31]:
yaml_path = "/content/legal_illegal.yaml"

yaml_text = f"""# legal vs illegal bird dataset
path: {DATASET_ROOT}

train: images
val: images

names:
  0: legal
  1: illegal
"""

with open(yaml_path, "w") as f:
    f.write(yaml_text)

print("📝 Wrote YAML to:", yaml_path)
print("-----")
print(yaml_text)


📝 Wrote YAML to: /content/legal_illegal.yaml
-----
# legal vs illegal bird dataset
path: /content/datasets/legal_illegal

train: images
val: images

names:
  0: legal
  1: illegal



In [33]:
model = YOLO("yolov8n.pt")  # nano model -> small & fast

results = model.train(
    data=yaml_path,
    epochs=100,
    imgsz=416,
    batch=16,
    device=0,         # GPU in Colab; use "cpu" if no GPU
    optimizer="AdamW",
    lr0=0.001,
    lrf=0.05,
    cos_lr=True,
    patience=100,
    mosaic=1.0,
    mixup=0.1,
    copy_paste=0.1,
    close_mosaic=10,
    amp=True,         # mixed precision for speed
    project="/content/runs",
    name="legal_illegal_yolov8n",
    exist_ok=True,
    save=True,
    save_period=-1,
    cache=False,
    plots=True,
)

print("✅ Training finished")


Ultralytics 8.3.231 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/legal_illegal.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.05, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=legal_illegal_yolov8n, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, perspecti

In [36]:
best_path = "/content/runs/legal_illegal_yolov8n/weights/best.pt"
print("⭐ Best model at:", best_path)

from google.colab import files
files.download(best_path)


⭐ Best model at: /content/runs/legal_illegal_yolov8n/weights/best.pt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>